# Getting Started with Julia in Colab/Jupyter
You can either run this notebook in Google Colab, or using Jupyter on your own machine.

## Running on Google Colab
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia (the Jupyter kernel for Julia) and other packages. You can update `JULIA_VERSION` and the other parameters, if you know what you're doing. Installation takes 2-3 minutes.
3. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the _Checking the Installation_ section.

* _Note_: If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 and 3.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.6.0 on the current Colab Runtime...
2021-12-05 11:25:02 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz [112838927/112838927] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system (if you ever ask for help or file an issue about Julia, you should always provide this information).

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


# Imports

In [2]:
using LinearAlgebra
using Statistics
using Dates

# How to get all the dates corresponding to the month of July 2016?

In [3]:
dr = Date("2016-07-01"):Day(1):Date("2016-07-31")

Date("2016-07-01"):Day(1):Date("2016-07-31")

In [4]:
# The above is probably all you need but if if do need to get the dates individually then this works
dr = [Date("2016-07-01"):Day(1):Date("2016-07-31");]

31-element Vector{Date}:
 2016-07-01
 2016-07-02
 2016-07-03
 2016-07-04
 2016-07-05
 2016-07-06
 2016-07-07
 2016-07-08
 2016-07-09
 2016-07-10
 2016-07-11
 2016-07-12
 2016-07-13
 ⋮
 2016-07-20
 2016-07-21
 2016-07-22
 2016-07-23
 2016-07-24
 2016-07-25
 2016-07-26
 2016-07-27
 2016-07-28
 2016-07-29
 2016-07-30
 2016-07-31

# How to compute ((A+B)*(-A/2)) in place (without copy)?

In [5]:
A = ones(3)
B = 2ones(3)

(A + B) .* (-A / 2)

3-element Vector{Float64}:
 -1.5
 -1.5
 -1.5

# Extract the integer part of a random array of positive numbers?

In [6]:
Z = rand(0:0.01:100,20)
make_int(x) = trunc(Int,x)
make_int.(Z)

20-element Vector{Int64}:
  2
 50
 77
 35
 93
  5
 79
 25
 86
 43
  8
 62
 75
 74
 34
 47
 49
 76
 39
 18

# Create a 5x5 matrix with row values ranging from 0 to 4 

In [7]:
A = fill([0:4;], 5)
# This is the solution if you want a array of arrays

5-element Vector{Vector{Int64}}:
 [0, 1, 2, 3, 4]
 [0, 1, 2, 3, 4]
 [0, 1, 2, 3, 4]
 [0, 1, 2, 3, 4]
 [0, 1, 2, 3, 4]

In [8]:
# The solution for this particular problem
A = [0 1 2 3 4]
repeat(A, 5)

5×5 Matrix{Int64}:
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4

In [9]:
# This is a more generic solution but slightly more involved
repeat(reshape([0:4;], 1, :), 5)
# Lets break down as so 
# * [0:4;] creates a vector [0,1,2,3,4], by default Julia makes a column vector
# * reshape([0:4;], 1, :) converts to 1x5 row vector from the column vector
# * The repeat function basically copies it five times

5×5 Matrix{Int64}:
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4

# Create a vector of size 10 with values ranging from 0 to 1, both excluded


In [10]:
Z = [rand(Float64) for i in 1:10]
# Most of the time this will work. I don't really see the need to actually exclude 0 as well from this code but if you do, then

10-element Vector{Float64}:
 0.19000787831694477
 0.6039444959114542
 0.08872983241089205
 0.1723994800162112
 0.9693130473792961
 0.8673575670727769
 0.3445378747290615
 0.8450938033005313
 0.5032069225806093
 0.3074776052360437

In [11]:
Z = [prevfloat(1.0) * (1 - rand(Float64)) for i in 1:10]
# Again its pretty extremely unlikely you'd actually need it but if you do, then you please read this discussion as well before going in https://discourse.julialang.org/t/how-to-create-a-random-uniform-distribution-between-but-excluding-0-and-10

10-element Vector{Float64}:
 0.3078683099324479
 0.5795611647015025
 0.3333871555760725
 0.158948176245669
 0.8328974355511917
 0.36319353274695215
 0.6579809742224746
 0.6023166244809838
 0.0291346680950848
 0.42114860955266903

# Create a random vector of size 10 and sort it

In [12]:
Z = rand(Int,10)
sort!(Z)

10-element Vector{Int64}:
 -7625865846929508973
 -4916795749092883684
 -4416856756873836290
 -2676359302979412501
 -2627430425537989624
  1654630053374251826
  3024451634057404757
  4573781989930118390
  5606720548413423782
  9028789117011591766

# Consider two random array A and B, check if they are equal

In [13]:
A = rand(1:5, 10)
B = rand(1:5, 10)

# To check if the arrays are completely equal
A == B

false

In [14]:
A = rand(1:5, 10)
B = rand(1:5, 10)
A,B

([4, 2, 5, 4, 5, 2, 4, 5, 4, 3], [2, 1, 3, 4, 2, 4, 4, 4, 5, 5])

In [15]:
# To check if the elements in the arrays are equal
A .== B

10-element BitVector:
 0
 0
 0
 1
 0
 0
 1
 0
 0
 0

# Consider a random 10x2 matrix representing cartesian coordinates, convert them to polar coordinates


In [16]:
struct cartesian_point
    X::Float32
    Y::Float32
end
points = [cartesian_point(rand(-900:0.01:900), rand(-900:0.01:900)) for _ = 1:10]

10-element Vector{cartesian_point}:
 cartesian_point(51.79f0, -125.64f0)
 cartesian_point(-358.75f0, -116.55f0)
 cartesian_point(-670.25f0, 850.11f0)
 cartesian_point(-43.68f0, -286.48f0)
 cartesian_point(-258.89f0, -108.05f0)
 cartesian_point(-31.74f0, 673.23f0)
 cartesian_point(213.59f0, 58.86f0)
 cartesian_point(293.91f0, -44.73f0)
 cartesian_point(-474.21f0, -777.58f0)
 cartesian_point(471.99f0, 706.64f0)

In [17]:
points = [cartesian_point(rand(), rand()) for _ = 1:10]
Xpoints = [point.X for point in points]
Ypoints = [point.Y for point in points]

10-element Vector{Float32}:
 0.22506145
 0.038344212
 0.34813055
 0.53168046
 0.21243542
 0.4196197
 0.35155025
 0.2258663
 0.18064925
 0.95632094

In [18]:
radians(x, y) = sqrt(x^2 + y^2)
theta(x, y) = atan(y, x)

theta (generic function with 1 method)

In [19]:
radians.(Xpoints, Ypoints)
theta.(Xpoints, Ypoints)

10-element Vector{Float32}:
 0.5390877
 0.099790566
 0.47372916
 0.51067334
 0.4834638
 0.8787195
 1.3252045
 0.22264798
 0.31063473
 0.79051477

# Create random vector of size 10 and replace the maximum value by 0


In [22]:
Z = rand(Int8, 10)
print(Z)
Z[argmax(Z)]

Int8[72, -118, 88, -112, 31, 108, -122, 13, 10, -58]

108

# Create a structured array with x and y coordinates covering the [0,1]x[0,1] area


In [23]:
Z = (X = collect(LinRange(0, 1, 5)), Y = collect(LinRange(0, 1, 5)))

(X = [0.0, 0.25, 0.5, 0.75, 1.0], Y = [0.0, 0.25, 0.5, 0.75, 1.0])

# Given two arrays, X and Y, construct the Cauchy matrix C (Cᵢⱼ = 1 ÷ (xᵢ - yⱼ))

In [39]:
X = rand(Float32, 10)
Y = rand(Float32, 10)

10-element Vector{Float32}:
 0.5028175
 0.8061949
 0.02918017
 0.98973334
 0.598259
 0.9094815
 0.42492604
 0.12290466
 0.37730277
 0.9200938

In [40]:
f(i, j) = 1 ÷ (i - j)
C = (f(x, y) for x in X, y in Y)
det(collect(C))

-2.7526134f12